In [1]:
import pdfplumber

with pdfplumber.open("./rheem.pdf") as pdf:
    for page in pdf.pages:
        for table in page.extract_tables():
            print(table)

[['', '', '', None, '', None, '', '', ''], [None, '', None, '', '', '', '', None, None], ['UA18AZ, R/UP18AZ, SA18AZ, SP18AZ, FRA18AZ, FRP18AZ', '', None, '', '', '', '', None, None], ['UA16AZ, R/UP16AZ, SA16AZ, SP16AZ, FRA16AZ, FRP16AZ', '', None, '', '', '', '', None, None], ['16AZ, RA15AZ, RA13NZ, RP15AZ, RP14AZ, SA15AZ, SA14AZ, SA13NZ, SD16AZ,\n15AZ, SP14AZ, FRA15AZ, FRA14AZ, FRA13NZ, FRD16AZ, FRP15AZ, FRP14AZ', '', None, '', '', '', '', None, None]]
[['', '', '', None], [None, '', None, ''], ['', '', None, ''], ['', '', None, '']]
[['', '', None, '', None, None, None, ''], ['', '', '', '', None, '', None, ''], ['', '', '', '', '', '', '', ''], ['', '', '', '', None, '', None, ''], ['', '', '', '', None, '', None, ''], ['', '', '', '', None, '', None, '']]
[['', '', None], [None, '', ''], ['', '', '']]
[['', '', '', None], [None, '', None, '']]


In [3]:
import subprocess

output = subprocess.Popen(["./pdftotext.exe", "goodman1.pdf", "-"], stdout=subprocess.PIPE).communicate()[0]

In [22]:
model_numbers = []
warranty_periods = []
for line in output.decode('windows-1252').split('\r\n'):
    line_lower = line.lower()
    if 'models' in line_lower:
        model_numbers.append(line)
    if 'year' in line_lower:
        warranty_periods.append(line)

In [25]:
set(model_numbers)

{'Models: GSXC, GSXV, GSZC, GSZV, GVXC, GVZC'}

In [27]:
print('\n'.join(warranty_periods))

How Long Does Warranty Coverage Last? Registration is not required to obtain warranty coverage, but registration entitles the owner to the Registered Additional Term Warranty referred to in the following paragraph. If the unit is not registered, the warranty on all parts lasts for a period of up to 5 YEARS after the installation date (the "Initial Term Warranty").
Except for units installed in Texas or Florida, the Registered Additional Term Warranty lasts for as long as the original registered owner or his or her spouse ("registered owner") own and reside in the residence in which the unit was originally installed, for (1) the lifetime of the registered owner for the compressor and (2) a period of up to 10 YEARS after the installation date for all other parts. For units installed in Texas or Florida, the Registered Additional Term Warranty on the compressor and all other parts lasts for a period of up to 10 YEARS after the installation date.
What Will Daikin Do To Correct Problems? Fa

In [7]:
"""
This code sample shows Prebuilt Layout operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = "https://westus2.api.cognitive.microsoft.com/"
key = "{Azure Document Intelligent Key}"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

with open("./goodman1.pdf", "rb") as open_pdf_file:
    poller = document_analysis_client.begin_analyze_document("prebuilt-layout", open_pdf_file)
result = poller.result()

content = [line.content for page in result.pages for line in page.lines]

#for table_idx, table in enumerate(result.tables):
#    print(
#        "Table # {} has {} rows and {} columns".format(
#        table_idx, table.row_count, table.column_count
#        )
#    )
        
#    for cell in table.cells:
#        print(
#            "...Cell[{}][{}] has content '{}'".format(
#            cell.row_index,
#            cell.column_index,
#            cell.content.encode("utf-8"),
#            )
#        )

#print("----------------------------------------")



In [8]:
'\n'.join(content)

'Goodman®\nAir Conditioning & Heating\nLIMITED WARRANTY\nModels: GSXC, GSXV, GSZC, GSZV, GVXC, GVZC\nExcept as otherwise provided in this document, if this heating\nor air conditioning unit has not been properly registered, all\nreferences in this document to "the warranty," "this warranty"\nor any similar construction, refer solely and exclusively to the\n· Units that are installed in buildings other than owner-\noccupied residences, such as non-residential buildings or\nresidences not occupied by the owner.\nInitial Term Warranty (defined below). If this heating or air\nWhat Problems Does This Warranty Cover?\nconditioning unit has been properly registered (and any other\napplicable conditions set forth in this document are satisfied),\nall such references refer to the Initial Term Warranty together\nThis warranty covers defects in materials and workmanship\nthat appear under normal use and maintenance.\nwith the Registered Additional Term Warranty (defined\nOther Warranties\nbelow).

In [12]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

llm = AzureChatOpenAI(azure_endpoint = "{OpenAPI URL}",
            openai_api_version = "2023-05-15",
            openai_api_key = '{OpenAPI key}', 
            deployment_name = 'gpt-35-long',
            temperature = 0.0)

In [14]:
messages = [SystemMessage(content="You are helping extract information from a warranty document for an HVAC/plumbing/electrical equipment."),
            HumanMessage(content="Please list all the model numbers and their warranty periods in the document. Write one model number per line. On each line, write model number : a single number representing the number of years in the warranty. If there are multiple warranty periods for a model, pick the one for registered, original owner, residential application. Pick the shortest one if there are multiple components in a model."),
    HumanMessage(content='\n'.join(content))]
llm(messages)

AIMessage(content='GSXC: 5\nGSXV: 5\nGSZC: 5\nGSZV: 5\nGVXC: 5\nGVZC: 5')